<a href="https://colab.research.google.com/github/iued-uni-heidelberg/corpustools/blob/main/S101lemKA_corrections_v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processing pipeline for DAAD project
## Stage 1: Lemmatization (Georgian) with corrections (manual annotation)

# Georgian 'random' corpus 
- (collected in SketchEngine using non-topical high-frequent keywords from the top 1000 of the wikipedia list)
- we try first on the corpus where corrections have been made for 3k Georgian most frequent word froms

In [1]:
!wget https://heibox.uni-heidelberg.de/f/362c24e311104c9aa35b/?dl=1
!mv index.html?dl=1 georgianrandom02.txt

--2023-01-30 07:37:17--  https://heibox.uni-heidelberg.de/f/362c24e311104c9aa35b/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/fd2065f1-80e6-4207-b603-1a622783ef54/georgianrandom02.txt [following]
--2023-01-30 07:37:18--  https://heibox.uni-heidelberg.de/seafhttp/files/fd2065f1-80e6-4207-b603-1a622783ef54/georgianrandom02.txt
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 192425594 (184M) [text/plain]
Saving to: ‘index.html?dl=1’

index.html?dl=1     100%[===================>] 183.51M  14.9MB/s    in 12s     

2023-01-30 07:37:30 (15.6 MB/s) - ‘index.html?dl=1’ saved [192425594/192425594]



In [ ]:
%%bash
# installing TreeTagger (en, de, ka)
mkdir treetagger
cd treetagger
# Download the tagger package for your system (PC-Linux, Mac OS-X, ARM64, ARMHF, ARM-Android, PPC64le-Linux).
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz
tar -xzvf tree-tagger-linux-3.2.4.tar.gz
# Download the tagging scripts into the same directory.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tagger-scripts.tar.gz
gunzip tagger-scripts.tar.gz
# Download the installation script install-tagger.sh.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/install-tagger.sh
# Download the parameter files for the languages you want to process.
# list of all files (parameter files) https://cis.lmu.de/~schmid/tools/TreeTagger/#parfiles
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/english.par.gz
sh install-tagger.sh
cd ..
sudo pip install treetaggerwrapper
# changing options: no-unknown, sgml, lemma
mv /content/treetagger/cmd/tree-tagger-english /content/tree-tagger-english0
awk '{ if (NR == 9) print "OPTIONS=\"-token -lemma -sgml -no-unknown\""; else print $0}' /content/tree-tagger-english0 > /content/treetagger/cmd/tree-tagger-english
chmod a+x ./treetagger/cmd/tree-tagger-english
# downloading German and Georgian 
wget https://heibox.uni-heidelberg.de/f/ec8226edebb64a359407/?dl=1
mv index.html?dl=1 /content/treetagger/lib/german-utf8.par
wget https://heibox.uni-heidelberg.de/f/9183090d2bdb41e09055/?dl=1
mv index.html?dl=1 /content/treetagger/lib/georgian.par
wget https://heibox.uni-heidelberg.de/f/9cafab0509d64ed1ac4b/?dl=1
mv index.html?dl=1 /content/treetagger/cmd/tree-tagger-georgian2
cp /content/treetagger/cmd/tree-tagger-georgian2 /content/treetagger/cmd/tree-tagger-georgian
# German2 = -no-unknown 
# note: tree-tagger-german will not work, as parameter files have not been downloaded, only use tree-tagger-german2 with utf8 encoding
wget https://heibox.uni-heidelberg.de/f/acb9b8a2fa4f40e08f8a/?dl=1
mv index.html?dl=1 /content/treetagger/cmd/tree-tagger-german2
chmod a+x /content/treetagger/cmd/tree-tagger-georgian2
chmod a+x /content/treetagger/cmd/tree-tagger-german2

In [ ]:
%%bash
wget https://heibox.uni-heidelberg.de/f/a6f7f36f175942ccad0a/?dl=1
mv index.html?dl=1 /content/treetagger/cmd/tree-tagger-georgian
chmod a+x /content/treetagger/cmd/tree-tagger-georgian

# cp /content/treetagger/cmd/tree-tagger-georgian2 /content/treetagger/cmd/tree-tagger-georgian
wget https://heibox.uni-heidelberg.de/f/e9010b0f3e7649ef9552/?dl=1
mv index.html?dl=1 georgianrandom--unknown-frq-all.tsv

In [4]:
# analysing German file 
# !./treetagger/cmd/tree-tagger-german2 /content/udhr/udhr_kat_v02.txt >/content/udhrTT/udhr_kat_vert.txt
!./treetagger/cmd/tree-tagger-georgian georgianrandom02.txt >georgianrandom02.vert

	reading parameters ...
	tagging ...
11301000	 finished.


In [ ]:
!head -n 50 georgianrandom02.vert


In [6]:
!wc georgianrandom02.vert

 11301824  33112747 394150707 georgianrandom02.vert


In [7]:
## how to print counter in AWK:
## https://stackoverflow.com/questions/67901330/awk-how-to-print-the-last-value-of-a-counter

# !awk -F '\t' '(NF==3){printf "%s ", $3; if(FNR % 10000 == 0){printf "\n"}}' < /content/udhrTT/udhr_deu_1996_vert.txt >/content/udhrTT/udhr_deu_1996_lem.txt

# if we need <p> tags for paragraphs:
# !awk -F '\t' '(NF==3){printf "%s ", $3}(NF!=3){printf "\n%s\n", $0}' < /content/udhrTT/udhr_deu_1996_vert.txt >/content/udhrTT/udhr_deu_1996_lem.txt
# if we do not need paragraph tags
# !awk -F '\t' '(NF==3){printf "%s ", $3}(NF!=3){printf "\n"}' < /content/udhrTT/udhr_kat_vert.txt >/content/udhrTT/udhr_kat_lem.txt
!awk -F '\t' '($3=="<unknown>"){j++; printf "%s\n", $0} END{print j+0  > "/dev/stderr" }' <georgianrandom02.vert >georgianrandom02-unknown.txt

2123093


In [8]:
cCoverage = 2123093 / 11301824
print(cCoverage)

0.18785401365301743


## Improving coverage with annotated word forms
- annotations are taken from `georgianrandom--unknown-frq-all.tsv'


In [9]:
import os, sys, re
DCorrectionsPoS = {}
DCorrectionsLem = {}
with open("georgianrandom--unknown-frq-all.tsv") as f:
    counter = 0
    for sline in f:
        counter += 1
        if counter == 1: continue # we skip the first line

        sline = sline.strip()
        LLine = sline.split('\t')
        try: SWord = LLine[1]
        except: SWord = ''

        try: SPoS = LLine[2]
        except: SPoS = ''

        try: SLemma = LLine[3]
        except: SLemma = ''

        try: SPoSCorrected = LLine[4]
        except: SPoSCorrected = ''
        if SPoSCorrected != '' and SWord != '':
            SPoS = SPoSCorrected
            DCorrectionsPoS[SWord] = SPoS

        if SWord != '' and SLemma != '': DCorrectionsLem[SWord] = SLemma
        # if SWord != '' and SPoS != '': DCorrectionsPoS[SWord] = SPoS

print(len(DCorrectionsPoS.items()))
print(len(DCorrectionsLem.items()))
FDictCorrected = open('dictLemmasCorrected.txt', 'w')
for key, value in DCorrectionsLem.items():
    FDictCorrected.write(f'{key}\t{value}\n')
FDictCorrected.flush()


416
2382


In [10]:
FOutVertCorrected = open('georgianrandom02corr.vert', 'w')
with open('georgianrandom02.vert') as FInVertUnknown:
    for SLine in FInVertUnknown:
        SLine = SLine.rstrip()
        LLine = SLine.split('\t')
        if len(LLine) != 3:
            FOutVertCorrected.write(f'{SLine}\n')
        else:
            SWord = LLine[0]
            SPoS = LLine[1]
            SLem = LLine[2]

            if SLem == '<unknown>':
                try: SLem = DCorrectionsLem[SWord]
                except: pass

                try: SPoS = DCorrectionsPoS[SWord]
                except: pass
            FOutVertCorrected.write(f'{SWord}\t{SPoS}\t{SLem}\n')
            
FOutVertCorrected.flush()


            

In [11]:
!wc georgianrandom02corr.vert

 11301824  33112747 398210244 georgianrandom02corr.vert


In [12]:
!awk -F '\t' '($3=="<unknown>"){j++; printf "%s\n", $0} END{print j+0  > "/dev/stderr" }' <georgianrandom02corr.vert >georgianrandom02corr-unknown.txt

1826952


In [13]:
cCoverage2 = 1826952 / 11301824
print(cCoverage2)

0.16165107508310164


## Running tagger / checking improvement on Specialised corpora
- downloading the corpora
- tagging
- checking baseline coverage
- checking improved coverage
- intersection of unknown words
- preparing datasets (1) intersections; (2) specialised for each corpus

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/dc5bcb4413aa42668130/?dl=1
!mv index.html?dl=1 specialisedCorpus.zip

In [ ]:
!unzip specialisedCorpus.zip

In [17]:
!wc specialised-corpora/*

   70620   596366 11660439 specialised-corpora/cFiktion.txt
   81314   529838 11302534 specialised-corpora/cNaturwissenschaft.txt
   63223   973857 21459641 specialised-corpora/cRechtswissenschaft.txt
  215157  2100061 44422614 total


In [32]:
# FName = 'ocrOutKA.txt'
# FNameOut = 'ocrLinesKA.txt'

def removeNewLines(FName, FNameOut):
    FIn = open(FName, 'r')
    FOut = open(FNameOut, 'w')

    for SLine in FIn:
        SLine = SLine.strip()
        if SLine == '': 
            FOut.write('\n\n')
            continue
        if SLine[-1] == '-':
            SLine2write = SLine[:-1]
            FOut.write(SLine2write)
            continue

        FOut.write(SLine + ' ')
    FOut.flush()
    return


In [25]:
!mkdir specialised-corpora-s01-lines/

In [34]:
removeNewLines('specialised-corpora/cFiktion.txt', 'specialised-corpora-s01-lines/cFiktion.txt')
removeNewLines('specialised-corpora/cNaturwissenschaft.txt', 'specialised-corpora-s01-lines/cNaturwissenschaft.txt')
removeNewLines('specialised-corpora/cRechtswissenschaft.txt', 'specialised-corpora-s01-lines/cRechtswissenschaft.txt')

In [ ]:
!head -n 10 specialised-corpora/cFiktion.txt

In [ ]:
!head -n 10 specialised-corpora-s01-lines/cFiktion.txt

In [ ]:
!head -n 50 specialised-corpora-s01-lines/cRechtswissenschaft.txt

In [39]:
!cat specialised-corpora-s01-lines/* >specialised-corpora-s01-all.txt

### preparing texts for tagging: removing new lines, etc.

In [40]:
!mkdir specialised-corpora-s02-vert/

In [ ]:
!./treetagger/cmd/tree-tagger-georgian specialised-corpora-s01-lines/cFiktion.txt >specialised-corpora-s02-vert/cFiktion.vert

In [ ]:
!./treetagger/cmd/tree-tagger-georgian specialised-corpora-s01-lines/cNaturwissenschaft.txt >specialised-corpora-s02-vert/cNaturwissenschaft.vert

In [ ]:
!./treetagger/cmd/tree-tagger-georgian specialised-corpora-s01-lines/cRechtswissenschaft.txt >specialised-corpora-s02-vert/cRechtswissenschaft.vert

In [ ]:
!./treetagger/cmd/tree-tagger-georgian specialised-corpora-s01-all.txt >specialised-corpora-s01-all.vert

### checking coverage on joint specialised corpus
- coverage of raw corpus
- coverage of corrections

In [45]:
!awk -F '\t' '($3=="<unknown>"){j++; printf "%s\n", $0} END{print j+0  > "/dev/stderr" }' <specialised-corpora-s01-all.vert >specialised-corpora-s01-all-unknown.vert

514895


In [46]:
!wc specialised-corpora-s01-all.vert
!wc specialised-corpora-s01-all-unknown.vert

 2567303  7701931 91404493 specialised-corpora-s01-all.vert
  514895  1544685 19910737 specialised-corpora-s01-all-unknown.vert


In [47]:
cCoverage3 = 514895 / 2567303
print(cCoverage3)

0.20055871862417488


## end: Specialised corpora

## Preparing datasets from further improvement
- what is still `unknown'


In [ ]:
!head -n 50 georgianrandom02-unknown.txt


In [ ]:
!tail -n 20 georgianrandom02-unknown.txt

In [ ]:
D = {}
with open("georgianrandom02-unknown.txt", 'r') as f:
    for line in f:
        line = line.rstrip()
        try:
            D[line] +=1
        except:
            D[line] = 1

fo = open("georgianrandom02-unknown-frq.txt", 'w')
ICorpusLen = 11301824
IUnknown = 2123093
FCoverage = IUnknown/ICorpusLen
print('percent Unknown TT: ', FCoverage)
IRestUnknown = IUnknown
c = 0
for key, val in sorted(D.items(), key=lambda item: item[1], reverse=True):
    c+=1
    IRestUnknown = IRestUnknown - val
    FRestCoverage = IRestUnknown/ICorpusLen
    fo.write(str(c) + '\t' + key + '\t' + str(val) + '\t' + str(FRestCoverage) + '\n')
    

percent Unknown TT:  0.18785401365301743


In [ ]:
!head -n 50 georgianrandom02-unknown-frq.txt

## Selecting 3 sets of 1k words for correction (random corpus)



In [ ]:
import sys, os, re

FOut1 = open('georgianrandom02-unknown-frq-p1of3.txt', 'w')
FOut2 = open('georgianrandom02-unknown-frq-p2of3.txt', 'w')
FOut3 = open('georgianrandom02-unknown-frq-p3of3.txt', 'w')
c = 0
ICorpusLen = 11301824
IUnknown = 2123093
IRestUnknown = IUnknown
for key, val in sorted(D.items(), key=lambda item: item[1], reverse=True):
    c+=1
    IRestUnknown = IRestUnknown - val
    FRestCoverage = IRestUnknown/ICorpusLen
    if(c + 2) % 3 == 0:
        FOut1.write(str(c) + '\t' + key + '\t' + str(val) + '\t' + str(FRestCoverage) + '\n')
    elif(c + 1) % 3 == 0:
        FOut2.write(str(c) + '\t' + key + '\t' + str(val) + '\t' + str(FRestCoverage) + '\n')
    elif c % 3 == 0:
        FOut3.write(str(c) + '\t' + key + '\t' + str(val) + '\t' + str(FRestCoverage) + '\n')
    else:
        sys.stdout.write('error: \t' + str(c) + '\t' + key + '\t' + str(val) + '\t' + str(FRestCoverage) + '\n')

## GIP DAAD corpus
### checking coverage

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/1981a35452db40c583c0/?dl=1
!mv index.html?dl=1 select-kat.tgz

In [ ]:
!tar xvzf select-kat.tgz

In [ ]:
import os
FOut = open('georgian-corp-gip-v01.txt', 'w')
for root, dirs, files in os.walk("./select-kat"):
    path = root.split(os.sep)
    print((len(path) - 1) * '---', os.path.basename(root))
    for file in files:
        print(len(path) * '---', file)
        FOut.write('<doc filename="' + file + '">\n')
        SFilePath = os.path.join(root,file)
        FIn = open(SFilePath, 'r')
        SIn = FIn.read()
        FOut.write(SIn)
        FOut.write('</doc>\n\n')

FOut.flush()
FOut.close()



In [ ]:
!wc georgian-corp-gip-v01.txt

   44448   892632 19802190 georgian-corp-gip-v01.txt


In [ ]:
!./treetagger/cmd/tree-tagger-georgian georgian-corp-gip-v01.txt >georgian-corp-gip-v01.vert

	reading parameters ...
	tagging ...
1086000	 finished.


In [ ]:
!wc georgian-corp-gip-v01.vert

 1086727  3260172 40704147 georgian-corp-gip-v01.vert


In [ ]:
!awk -F '\t' '($3=="<unknown>"){j++; printf "%s\n", $0} END{print j+0  > "/dev/stderr" }' <georgian-corp-gip-v01.vert >georgian-corp-gip-v01-unknown.txt

222297


In [ ]:
ICorpusLen = 1086727
IUnknown = 222297
FCoverage = IUnknown/ICorpusLen
print('percent Unknown TT: ', FCoverage)

percent Unknown TT:  0.2045564341366323


In [ ]:
E = {}
with open("georgian-corp-gip-v01-unknown.txt", 'r') as f:
    for line in f:
        line = line.rstrip()
        try:
            E[line] +=1
        except:
            E[line] = 1

In [ ]:
## checking coverage on georgianrandom02 corpus
IRestUnknown = IUnknown
c = 0
CFound = 0
fo = open("georgian-corp-gip-v01-unknown-from-georgianrandom02-frq.txt", 'w')
for key, val in sorted(D.items(), key=lambda item: item[1], reverse=True):
    c+=1
    BFound = False
    try: 
        val2 = E[key]
        BFound = True
        CFound += 1
    except:
        val2 = 0

    IRestUnknown = IRestUnknown - val2
    FRestCoverage = IRestUnknown/ICorpusLen
    fo.write(str(c) + '\t' + key + '\t' + str(val) + '\t' + str(BFound) + '\t' + str(CFound) + '\t' + str(FRestCoverage) + '\n')



# Optional dataset (not used)
## Example: Universal declaration of human rights (

In [ ]:
# Downloading UDHR
!wget https://unicode.org/udhr/assemblies/udhr_txt.zip


In [ ]:
%%bash
mkdir udhr
mv udhr_txt.zip udhr
cd udhr/
unzip udhr_txt.zip

In [ ]:
%%bash
# put paragraph tags
awk '{print "<p>\n"$0 ; print "</p>"}' /content/udhr/udhr_kat.txt >/content/udhr/udhr_kat_v02.txt